# Ingest from Source
### I usually have all the functions to interact with the data source in the _INIT_ including Authentication, token refresh, and secret retrieval

In [0]:
#create load function
def ingestRefData(fileName):

    # File location and type
    file_location = f"/FileStore/tables/MDE/{fileName}"
    file_type = "csv"

    # CSV options
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = ","

    # The applied options are for CSV files. For other file types, these will be ignored.
    df = spark.read.format(file_type) \
        .option("inferSchema", infer_schema) \
        .option("header", first_row_is_header) \
        .option("sep", delimiter) \
        .load(file_location)

    tableName = fileName[:-4]
    #I print to grab the names of files for review- when pushing to PROD, the manual interventions go away
    print(f'{tableName} created into spark dataframe')

    #turn into delta view for temp storage- no need for these to be in perm/table storage
    df.createOrReplaceTempView(tableName)

In [0]:
def IngestClinicalFiles(urlList):
    # Initialize the schema and DataFrame
    schema = None
    df = None

    # Iterate through the list; for optimization look at multiprocessing
    for url in urlList:
        res = requests.get(url)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".xml") as temp_file:
            temp_file.write(res.content)
            temp_file_path = temp_file.name

        # Move the temporary file to DBFS
        dbfs_path = f"/dbfs/tmp/{temp_file.name.split('/')[-1]}"
        shutil.move(temp_file_path, dbfs_path)

        # Load the XML file from DBFS
        df2 = (spark.read
            .format('xml')
            .option('rowTag', 'ClinicalDocument')
            #.option('excludeAttribute', 'true')  # Exclude attributes to avoid column name conflicts
            .schema(inferred_schema)
            .load(f"dbfs:/tmp/{temp_file.name.split('/')[-1]}"))
        
        if df is None:
            df = df2
        else:
            df = df.unionByName(df2, allowMissingColumns=True)
        print(f'Added {x}')

    display(df)
    return df

# These functions flatten, explode, and normalize the pyspark dataframe schema
### Doesn't work great with HL7 xml parsing

In [0]:
from pyspark.sql.functions import explode, col

def flatten_schema(schema, prefix=None):
    fields = []
    for field in schema.fields:
        name = f"{prefix}.{field.name}" if prefix else field.name
        if isinstance(field.dataType, StructType):
            fields += flatten_schema(field.dataType, prefix=name)
        else:
            fields.append(name)
    return fields

def explode_df(df):
    for (name, dtype) in df.dtypes:
        if "array" in dtype:
            df = df.withColumn(name, explode(name))
    return df

from pyspark.sql.functions import explode, col

def is_flat(df):
    for (_, dtype) in df.dtypes:
        if "array" in dtype or "struct" in dtype:
            return False
    return True

def flatten_nest_df(df):
    keep_going = True
    while(keep_going):
        fields = flatten_schema(df.schema)
        new_fields = [item.replace(".", "_").lower() for item in fields]
        df = df.select(fields).toDF(*new_fields)
        df = explode_df(df)
        if is_flat(df):
            keep_going = False
    return df
#dfx = flatten_nest_df(df) Input pyspark dataframe of xml and run